# Training Pipeline

In this notebook, we will do the following tasks

1. Get the data from the feature store
2. Preprocess the data
3. Train the model
4. Evaluate the model
5. Register the model to model registry


In [1]:
# Import the required libraries
import os
import hopsworks
import numpy as np
import pandas as pd

from dotenv import load_dotenv

# Load the .env file 
load_dotenv()

# Get the envrioment variables
hopsworks_api_key = os.getenv("HOPSWORKS_API_KEY")
 

In [2]:
# login hopsworks and get the features group
project = hopsworks.login(api_key_value=str(hopsworks_api_key))
fs = project.get_feature_store()



2025-02-21 21:13:11,971 INFO: Initializing external client
2025-02-21 21:13:11,971 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-21 21:13:15,109 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212597


## Feature View


### Feature Selection


In [3]:
amazon_fg = fs.get_feature_group("amazon_stock_prices", version=1)

In [4]:
# Select features for training data
selected_features = amazon_fg.select(["date", "open", "high", "close", "low"])

# View the first 5 rows of selected features
selected_features.show(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.81s) 


,date,open,high,close,low
0,2024-03-19 17:00:00+00:00,175.710,175.985,175.540,175.33
1,2024-05-24 14:00:00+00:00,181.570,182.165,181.350,181.04
2,2023-06-14 16:00:00+00:00,126.450,126.460,126.305,126.07
3,2023-02-14 16:00:00+00:00,98.470,98.855,97.970,97.53
4,2023-07-17 14:00:00+00:00,134.075,134.710,134.065,133.61


### Feature View Creation


In [5]:
# Get or create feature view
amazon_fv = fs.get_or_create_feature_view(
    name= "amazon_fv", 
    version=1,
    query = selected_features,
)

## Training Dataset Creation


In [6]:
# Get the data to calculate date ranges
df = amazon_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.77s) 


In [7]:
# Sort the values according to the date
df = df.sort_values("date").set_index("date")

df.head()

,close,high,low,open,id
date,,,,,
2022-06-07 14:00:00+00:00,123.350,124.090,121.55,122.440,2022-06-07 14:00:00
2022-06-07 15:00:00+00:00,122.420,123.375,121.53,123.360,2022-06-07 15:00:00
2022-06-07 16:00:00+00:00,121.670,122.500,121.03,122.500,2022-06-07 16:00:00
2022-06-07 17:00:00+00:00,122.085,122.475,121.40,121.680,2022-06-07 17:00:00
2022-06-07 18:00:00+00:00,122.190,122.810,121.95,122.085,2022-06-07 18:00:00


In [8]:
def get_fractional_dates(column: pd.Series):
    total_length = len(column)
    
    idx_0 = 0  # First index
    idx_70 = int(0.7 * total_length)
    idx_85 = int(0.85 * total_length)
    idx_last = total_length - 1  # Last index

    def extract_date(idx):
        return str(column.index[idx]).split(" ")[0]  # Convert to string and get date part

    train_start = extract_date(idx_0)
    val_start = extract_date(idx_70)
    test_start = extract_date(idx_85)

    train_end = extract_date(idx_70 - 1)  # One day before val_start
    val_end = extract_date(idx_85 - 1)  # One day before test_start
    test_end = extract_date(idx_last)  # Last available date

    return train_start, train_end, val_start, val_end, test_start, test_end

In [9]:
train_start, train_end, val_start, val_end, test_start, test_end = get_fractional_dates(df)
train_start, train_end, val_start, val_end, test_start, test_end

('2022-06-07',
 '2024-04-30',
 '2024-04-30',
 '2024-09-24',
 '2024-09-25',
 '2025-02-20')

#### Split the data into train, val and test splits


In [10]:
train, val, test, _, _, _ =  amazon_fv.train_validation_test_split(
    train_start=train_start,
    train_end = train_end,
    val_start = val_start,
    val_end = val_end,
    test_start=test_start,
    test_end = test_end    
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.84s) 
2025-02-21 21:13:32,928 WARNING: VersionWarning: Incremented version to `2`.



In [11]:
# Sort and remove the date column to ensure all data is in the same format
train = train.sort_values("date").drop("date", axis=1)
val = val.sort_values("date").drop("date", axis=1)
test = test.sort_values("date").drop("date", axis=1)

### Creating TorchDataset and DataLoaders


In [12]:
import torch
import comet_ml
from torch import nn
from torch.utils.data import DataLoader, Dataset


# Setup hyperparameters
hyper_params = {
    "window_size": 24,
    "forecast_steps" : 6,
    "batch_size" : 32,
    "hidden_size" : 128,
    "num_layers" : 2,
    "num_epochs" : 100,
    "learning_rate": 0.001
}

### Normalize the data

LSTM is sensitive to the scale of the input data. Here, we'll normalize the data to the range [0, 1] using MinMaxScaler.


In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)


In [14]:
def create_sequences(data: pd.DataFrame, window_size: int = hyper_params['window_size'], forecast_steps: int = hyper_params['forecast_steps']):
    # Initialize empty lists
    X, y = [], []
        
    # Loop through the dataset
    for i in range(len(data) - window_size - forecast_steps):
        X.append(data[i: i + window_size, :])
        
        # y values should start from the next value where X ended to forecast steps
        # close is the target column and is at the 3rd place
        y.append(data[i + window_size: i + window_size + forecast_steps, 2])
        
    return np.array(X), np.array(y)

In [15]:
# Test our data
X_train, y_train  = create_sequences(train)
X_val, y_val = create_sequences(val)
X_test, y_test = create_sequences(test)

In [16]:
# View the shapes of train, val and test splits
print(f"Shape of Training Data: {X_train.shape, y_train.shape}")
print(f"Shape of Validation Data: {X_val.shape, y_val.shape}")
print(f"Shape of Testing Data: {X_test.shape, y_test.shape}")

Shape of Training Data: ((2934, 24, 4), (2934, 6))
Shape of Validation Data: ((606, 24, 4), (606, 6))
Shape of Testing Data: ((600, 24, 4), (600, 6))


In [17]:
# Create Torch Dataset

class AmazonDataset(Dataset):
    """ Creates Amazon Stock Price Dataset """
    def __init__(self, X: np.ndarray,y: np.ndarray):
        super().__init__()  # Call the parent class constructor
        self.X = X
        self.y = y
        
    def __len__(self):
            return len(self.X)
        
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

In [18]:
# Create dataset and dataloaders
train_data = AmazonDataset(X_train, y_train)
val_data = AmazonDataset(X_val, y_val)
test_data = AmazonDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=hyper_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_data, batch_size=hyper_params['batch_size'], shuffle=False)
test_loader = DataLoader(test_data, batch_size=hyper_params['batch_size'], shuffle=False)


In [19]:
len(train_loader), len(val_loader), len(test_loader)

(92, 19, 19)

In [20]:
# check the shape of single batch

for X, y in train_loader:
    print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
    break

Shape of X: torch.Size([32, 24, 4]), Shape of y: torch.Size([32, 6])


## Modelling


In [21]:
# Setup Logging in Comet ml
from comet_ml.integration.pytorch import watch

# Set up the Comet experiment
load_dotenv()
comet_api_key = os.getenv("COMET_API_KEY")

# Login to comret
comet_ml.login(api_key=comet_api_key)



experiment = comet_ml.start(project_name='lstm-stock-price-prediction')
experiment.log_parameters(hyper_params)


COMET INFO: Valid Comet API Key saved in C:\Users\amuly\.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/amulyaprasanth/lstm-stock-price-prediction/7b30b0ad8a154149b60b4c2874882f39



2025-02-21 21:13:44,372 WARNING: DeprecationWarning: Please use dulwich.object_store.iter_tree_contents



In [22]:
# Create the model
class LSTMModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, num_layers: int, device:str = 'cpu'):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.device = device
        
		# LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
	# forward pass
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)
        
        out, (_, _) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
        


In [23]:
from torchinfo import summary

# Get the summary of the model
summary(LSTMModel(input_dim=X_train.shape[2], hidden_dim=hyper_params['hidden_size'], output_dim=6, num_layers=2, device='cuda'), input_size=(hyper_params['batch_size'], X_train.shape[1], X_train.shape[2]))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMModel                                [32, 6]                   --
├─LSTM: 1-1                              [32, 24, 128]             200,704
├─Linear: 1-2                            [32, 6]                   774
Total params: 201,478
Trainable params: 201,478
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 154.17
Input size (MB): 0.01
Forward/backward pass size (MB): 0.79
Params size (MB): 0.81
Estimated Total Size (MB): 1.61

In [24]:
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader

def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, num_epochs: int, device: str = 'cuda'):
    """
    Trains and validates a PyTorch model.

    Args:
        model (nn.Module): The neural network model to be trained.
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
        loss_fn (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        num_epochs (int): Number of epochs to train the model.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        None
    """
    # Move model to GPU if available
    model = model.to(device)

    watch(model)
    for epoch in tqdm(range(num_epochs), desc= 'Epochs: '):
        train_loss, test_loss = 0.0, 0.0
        
        ### Train time
        # Set the model to training mode
        model.train()
        
        # Iterate over the training data
        for X, y in tqdm(train_loader, desc='Training...', leave=False):
            # Move data to GPU if available
            X, y = X.to(device), y.to(device)

            # optimizer zero grad
            optimizer.zero_grad()
            
            # Do the forward pass
            outputs = model(X)
            
            # Calculate the loss
            loss = loss_fn(outputs, y)
            train_loss += loss.item()
            
            
            # loss backward
            loss.backward()
            
            # optimizer step
            optimizer.step()
    
        ### Test time
        # Set the model to evaluation mode
        model.eval()
        
        with torch.inference_mode():
            # Iterate over the validation data
            for X, y in tqdm(val_loader, desc='Evaluating...', leave=False):
                # Move data to GPU if available
                X, y = X.to(device), y.to(device)
                
                # Do the forward pass
                outputs = model(X)
                
                # Calculate the loss
                loss = loss_fn(outputs, y)
                test_loss += loss.item()

        train_loss /= len(train_loader)
        test_loss /= len(val_loader)

        # Log the train and test loss to Comet ML
        experiment.log_metric("train_loss", train_loss, epoch=epoch)
        experiment.log_metric("test_loss", test_loss, epoch=epoch)
        
        # Print the train and test loss
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:7f}, Test Loss: {test_loss:.7f}")

    # End the experiment
    experiment.end()

In [25]:
# Define the loss function and optimizer
model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=hyper_params['hidden_size'], output_dim=hyper_params['forecast_steps'], num_layers=hyper_params['num_layers'], device='cuda')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])

# Train the model with logging
train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=hyper_params['num_epochs'], device='cuda')

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training...:   0%|          | 0/92 [00:00<?, ?it/s]

Evaluating...:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.023673, Test Loss: 0.0025392


Training...:   0%|          | 0/92 [00:00<?, ?it/s]

...

Training...:   0%|          | 0/92 [00:00<?, ?it/s]

Evaluating...:   0%|          | 0/19 [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : electronic_tick_1014
COMET INFO:     url                   : https://www.comet.com/amulyaprasanth/lstm-stock-price-prediction/7b30b0ad8a154149b60b4c2874882f39
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_loss [100]  : (0.0005682111368514597, 0.002566312396849849)
COMET INFO:     train_loss [100] : (0.00039780466194434155, 0.02367323348311323)
COMET INFO:   Parameters:
COMET INFO:     batch_size     : 32
COMET INFO:     forecast_steps : 6
COMET INFO:     hidden_size    : 128
COMET INFO:     learning_rate  : 0.001
COMET INFO:     num_epochs     : 100
COMET INFO:     num_layers     : 2
COMET INFO:     window_size    : 24
COMET INFO:   

Epoch 100/100, Train Loss: 0.000404, Test Loss: 0.0005860


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
